In [17]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import time

def download_image(url, save_path):
    time.sleep(0.5)
    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(1024):
            file.write(chunk)

def scrape_images(query, num_images):
    query = query.replace(" ", "+")
    search_url = f"https://www.google.com/search?q={query}&source=lnms&tbm=isch"
    query = query.replace("+", "_")
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    image_urls = []
    for img in soup.findAll('img', class_='Q4LuWd'):
        if len(image_urls) == num_images:
            break
        try:
            img_url = img["data-src"]
        except KeyError:
            img_url = img["src"]
        if img_url.startswith("http"):
            image_urls.append(img_url)

    save_dir = os.path.join(os.getcwd(), 'Unprocessed', query)
    os.makedirs(save_dir, exist_ok=True)

    for i, url in enumerate(image_urls):
        filename = f"{i+1}.jpg"
        save_path = os.path.join(save_dir, filename)
        download_image(url, save_path)
#         print(f"Downloaded {filename}.")
    
    return save_dir

In [2]:
import cv2

def detect_and_crop_faces(image_path, dirname, filename):    
    # Load the Haar cascade face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    x, y, w, h = faces[0]
    # Calculate the coordinates of the bounding box
    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h

    # Crop the image around the face, maintaining color channels
    cropped_face = image[y1:y2, x1:x2].copy()

    # Save the cropped face as a new image
    file_jpg = f"{filename}.jpg"
    save_path = os.path.join(dirname, file_jpg)
    cv2.imwrite(save_path, cropped_face)
#     print(f"Saved {save_path}.")

In [13]:
import time
import random

def scrape_faces(query_list, num_images=10):
    folder_list = []
    
    for query in query_list:
        print(f"Querying {query}")
        # Use Timeout so that we don't flood the server
        time.sleep(random.randint(2, 4))
        # Scrape Image from google
        try:
            folder = scrape_images(query, num_images)
            # Save the folder name for future process
            folder_list.append(folder)
        except Exception as e:
            print(f"{query} Error with value: {e}")
            pass
        
        print("Done")
    
    return folder_list

In [4]:
def get_folder_paths(folder_path):
    folder_paths = []
    for root, dirs, files in os.walk(folder_path):
        for dir_ in dirs:
            path = os.path.join(root, dir_)
            folder_paths.append(path)
    return folder_paths

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

def remove_file(path):
    os.remove(path)
#     print(f"{path} Deleted")

def detect_faces(folder_list):
    
    for folder in folder_list:
        print(f"Processing {folder[-20:]}")
        
        try:
            files = get_file_paths(folder)
        except Exception as e:
            print(f"Get File Path for {folder} Error with value: {e}")
            continue
        
#         print(files)
        for i, filepath in enumerate(files):
            try:
                detect_and_crop_faces(filepath, folder, str(i+1))
                remove_file(filepath)
            except Exception as e:
                print(f"Detect and crop face for {filepath} Error with value: {e}")
        
        print("Done")

In [16]:
with open('list_politikus.txt', 'r') as f:
    list_politikus = f.readlines()
    list_politikus = [name.replace("\n", "") for name in list_politikus]

with open('list_komedian.txt', 'r') as f:
    list_komedian = f.readlines()
    list_komedian = [name.replace("\n", "") for name in list_komedian]
    
with open('list_singer.txt', 'r') as f:
    list_singer = f.readlines()
    list_singer = [name.replace("\n", "") for name in list_singer]
    
with open('list_atlit.txt', 'r') as f:
    list_atlit = f.readlines()
    list_atlit = [name.replace("\n", "") for name in list_atlit]
    

with open('list_youtuber.txt', 'r') as f:
    list_youtuber = f.readlines()
    list_youtuber = [name.replace("\n", "") for name in list_youtuber]

list_youtuber

['Ria Ricis',
 'Atta Halilintar',
 'Jess No Limit',
 'Baim Wong',
 'Deddy Corbuzier',
 'Reza Oktovian',
 'Edho Zell',
 'Chandra Liow',
 'Bayu Skak',
 'Windah Basudara',
 'Hanin Dhiya',
 'Andovi Da Lopez',
 'Jovial Da Lopez',
 'Eka Gustiwana',
 'Naisa Alifia Yuriza',
 'tanboy kun',
 'Jessica Jane',
 'Jerome Polin']

In [18]:
folders_politikus = scrape_faces(list_politikus, 20)
folders_politikus

Querying Susilo Bambang Yudhoyono
Done
Querying Joko Widodo
Done
Querying Prabowo Subianto
Done
Querying Megawati Soekarnoputri
Done
Querying Puan Maharani
Done
Querying Basuki Tjahaja Purnama
Done


['C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Susilo_Bambang_Yudhoyono',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Joko_Widodo',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Prabowo_Subianto',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Megawati_Soekarnoputri',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Puan_Maharani',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project 

In [19]:
folders_komedian = scrape_faces(list_komedian, 20)
folders_komedian

Querying Yono Bakrie
Done
Querying Gautama Shindu
Done
Querying Abdel Achrian
Done
Querying Raditya Dika
Done
Querying Pandji Pragiwaksono
Done
Querying Ge Pamungkas
Done
Querying Gilang Dirga
Done
Querying Arie Kriting
Done
Querying Ayu Dewi
Done
Querying Andhika Pratama
Done
Querying Gilang Bhaskara
Done
Querying David Nurbianto
Done
Querying Cak Lontong
Done
Querying Abdurrahim Arsyad
Done
Querying Mamat Alkatiri
Done
Querying Bintang Emon
Done


['C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Yono_Bakrie',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Gautama_Shindu',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Abdel_Achrian',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Raditya_Dika',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Pandji_Pragiwaksono',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset

In [20]:
folders_singer = scrape_faces(list_singer, 20)
folders_singer

Querying Agnez Mo
Done
Querying Afgansyah Reza
Done
Querying Ahmad Albar
Done
Querying Ahmad Dhani
Done
Querying Anggun
Done
Querying Ari Lasso
Done
Querying Ayu Ting Ting
Done
Querying Ashanty
Done
Querying Aurel Hermansyah
Done
Querying Benyamin Sueb
Done
Querying Brian Imanuel
Done
Querying Bunga Citra Lestari
Done
Querying Chrisye
Done
Querying Cinta Laura
Done
Querying Citra Scholastika
Done
Querying Dewi Lestari
Done
Querying Dewi Persik
Done
Querying Dewi Sandra
Done
Querying Didi Kempot
Done
Querying Ebiet G Ade
Done
Querying Elvy Sukaesih
Done
Querying Eros Djarot
Done
Querying Evie Tamala
Done
Querying Fatin Shidqia
Done
Querying Glenn Fredly
Done
Querying Gita Gutawa
Done
Querying Indah Dewi Pertiwi
Done
Querying Inul Daratista
Done
Querying Iwa K
Done
Querying Iwan Fals
Done
Querying Isyana Sarasvati
Done
Querying Julia Perez
Done
Querying Lyodra Ginting
Done
Querying Andriani Marshanda
Done
Querying Maudy Ayunda
Done
Querying Melky Goeslaw
Done
Querying Mike Mohede
Done
Qu

['C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Agnez_Mo',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Afgansyah_Reza',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Ahmad_Albar',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Ahmad_Dhani',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Anggun',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unproc

In [21]:
folders_atlit = scrape_faces(list_atlit, 20)
folders_atlit

Querying Anthony Sinisuka Ginting
Done
Querying Jonatan Christie
Done
Querying Shesar Hiren Rhustavito
Done
Querying Kevin Sanjaya Sukamuljo
Done
Querying Mohammad Ahsan
Done
Querying Hendra Setiawan
Done
Querying Fajar Alfian
Done
Querying Muhammad Rian Ardianto
Done
Querying Bagas Maulana
Done
Querying Muhammad Shohibul Fikri
Done
Querying Komang Ayu Cahya Dewi
Done
Querying Tasya Farahnailah
Done
Querying Siti Sarah Azzahra
Done
Querying Nita Violina Marwah
Done
Querying Jesita Putri Miantoro
Done
Querying Lanny Tria Mayasari
Done
Querying Tryola Nadia
Done
Querying Melani Mamahit
Done
Querying Wilda Siti Nurfadhila
Done
Querying Tisya Amalia Putri
Done
Querying Shintia Alliva Mauludina
Done
Querying Myrasuci Indriani
Done
Querying Megawati Hangestri P
Done
Querying Hany Budiarti
Done
Querying Arneta Putri Amelian
Done
Querying Mediol Stiovanny Yoku
Done
Querying Yulis Indahyani
Done
Querying Ernando Ari
Done
Querying Bagas Kaffa
Done
Querying Komang Teguh
Done
Querying Pratama Arha

['C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Anthony_Sinisuka_Ginting',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Jonatan_Christie',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Shesar_Hiren_Rhustavito',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Kevin_Sanjaya_Sukamuljo',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed\\Mohammad_Ahsan',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 

In [14]:
folders_youtuber = scrape_faces(list_youtuber, 20)
folders_youtuber

Querying Ria Ricis
Done
Querying Atta Halilintar
Done
Querying Jess No Limit
Done
Querying Baim Wong
Done
Querying Deddy Corbuzier
Done
Querying Reza Oktovian
Done
Querying Edho Zell
Done
Querying Chandra Liow
Done
Querying Bayu Skak
Done
Querying Windah Basudara
Done
Querying Hanin Dhiya
Done
Querying Andovi Da Lopez
Done
Querying Jovial Da Lopez
Done
Querying Eka Gustiwana
Done
Querying Naisa Alifia Yuriza
Done
Querying tanboy kun
Done
Querying Jessica Jane
Done
Querying Jerome Polin
Done


['C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Ria_Ricis',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Atta_Halilintar',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Jess_No_Limit',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Baim_Wong',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Deddy_Corbuzier',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Reza_Oktovian',
 'C:\\Users\\arifn\\OneDriveKuliah\\OneDrive

In [22]:
detect_faces(get_folder_paths("C:\\Users\\arifn\\OneDriveKuliah\\OneDrive - Institut Teknologi Sepuluh Nopember\\Kuliah\\Bangkit 2023\\Project Capstone\\Dataset Collection\\Unprocessed"))

Processing cessed\Abdel_Achrian
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Abdel_Achrian\10.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Abdel_Achrian\11.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Abdel_Achrian\12.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Abdel_Achrian\13.jpg Error with value: tuple index out of range
Detect and crop face for C:\User

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Andhika_Pratama\4.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Andhika_Pratama\5.jpg Error with value: tuple index out of range
Done
Processing d\Andriani_Marshanda
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Andriani_Marshanda\1.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Andriani_Marshanda\14.jpg Error with value: tuple index out of range
Detect and crop 

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Ashanty\12.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Ashanty\9.jpg Error with value: tuple index out of range
Done
Processing sed\Aurel_Hermansyah
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Aurel_Hermansyah\10.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Aurel_Hermansyah\2.jpg Error with value: tuple index out of range
Done
Processing Unprocessed\Ayu_Dew

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Basuki_Tjahaja_Purnama\11.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Basuki_Tjahaja_Purnama\13.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Basuki_Tjahaja_Purnama\16.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Basuki_Tjahaja_Purnama\2.jpg Error with value: tuple index out of range
Detect and crop face for C:\U

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Brian_Imanuel\20.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Brian_Imanuel\4.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Brian_Imanuel\5.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Brian_Imanuel\6.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - I

...

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Rizky_Ridho\2.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Rizky_Ridho\20.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Rizky_Ridho\3.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Rizky_Ridho\4.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut 

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Siti_Sarah_Azzahra\15.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Siti_Sarah_Azzahra\17.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Siti_Sarah_Azzahra\18.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Siti_Sarah_Azzahra\2.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDr

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tiara_Andini\5.jpg Error with value: tuple index out of range
Done
Processing d\Tisya_Amalia_Putri
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tisya_Amalia_Putri\1.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tisya_Amalia_Putri\10.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tisya_Amalia_Putri\11.jpg Error with value: tuple index out of range
Detect and crop

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tryola_Nadia\4.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tryola_Nadia\6.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tryola_Nadia\8.jpg Error with value: tuple index out of range
Done
Processing on\Unprocessed\Tulus
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Tulus\18.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\O

Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Yono_Bakrie\15.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Yono_Bakrie\17.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Yono_Bakrie\3.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut Teknologi Sepuluh Nopember\Kuliah\Bangkit 2023\Project Capstone\Dataset Collection\Unprocessed\Yono_Bakrie\4.jpg Error with value: tuple index out of range
Detect and crop face for C:\Users\arifn\OneDriveKuliah\OneDrive - Institut